# Homework 3 -- Cloropleth map of grants in Swiss universities

In [27]:
# imports and all that
import pandas as pd

### Data wrangling

In [30]:
grants = pd.read_csv("data/P3_GrantExport.csv", sep=';', index_col=0)
grants.size
grants.shape

(48574, 8)

We'll try to reduce the dataset by removing the entries that are not in CH and irrelevant columns.
We also remove data where location cannot be inferred, i.e. which are missing both the institute and the university

These changes can be safely persisted in the file, as it will not influence the data we work with.

In [31]:
grants = grants[grants.University.notnull()]
grants = grants.drop(axis=1, errors='ignore', labels=['Project Title', 'Project Title English',
            'Responsible Applicant', 'Funding Instrument', 'Funding Instrument Hierarchy', 'Keywords'])

# drop rows where location cannot be inferred
filter_no_location = ~((grants.University == "Nicht zuteilbar - NA") & (grants.Institution.isnull()))
grants = grants[filter_no_location]
grants.dtypes
grants.shape

(48574, 8)

In [32]:
# Sum entries in the Approved Amount column are not numeric and such are being dropped
grants['Approved Amount'] = pd.to_numeric(grants['Approved Amount'], errors='coerce')
grants = grants[grants['Approved Amount'].isnull() == False]
grants.shape

(48464, 8)

In [33]:
# Run if you want to update the csv
#grants.to_csv("data/P3_GrantExport.csv", sep=';') # keep the same sep

In [39]:
import re
def getLocationTagFromUni(s):
    return re.findall(r'\S+', s)[-1]

In [40]:
grants['Location Tag'] = grants['University'].apply(getLocationTagFromUni)
grants.head()

,Institution,University,Discipline Number,Discipline Name,Discipline Name Hierarchy,Start Date,End Date,Approved Amount,Location Tag
"﻿""Project Number""",,,,,,,,,
4,Faculté de Psychologie et des Sciences de l'Ed...,Université de Genève - GE,10104,Educational science and Pedagogy,"Human and Social Sciences;Psychology, educatio...",01.10.1975,30.09.1976,41022.0,GE
5,Kommission für das Corpus philosophorum medii ...,"NPO (Biblioth., Museen, Verwalt.) - NPO",10101,Philosophy,Human and Social Sciences;Linguistics and lite...,01.03.1976,28.02.1985,79732.0,NPO
6,Abt. Handschriften und Alte Drucke Bibliothek ...,Universität Basel - BS,10302,Swiss history,Human and Social Sciences;Theology & religious...,01.10.1975,30.09.1976,52627.0,BS
7,Schweiz. Thesauruskommission,"NPO (Biblioth., Museen, Verwalt.) - NPO",10303,Ancient history and Classical studies,Human and Social Sciences;Theology & religious...,01.01.1976,30.04.1978,120042.0,NPO
8,"Séminaire de politique économique, d'économie ...",Université de Fribourg - FR,10203,Economics,"Human and Social Sciences;Economics, law",01.01.1976,31.12.1978,53009.0,FR


In [66]:
# Grouped by 'University' and the total funds each gets 
grants_grouped_by_uni = grants.groupby(['Location Tag'])
grants_grouped_by_uni = grants_grouped_by_uni.agg({'Approved Amount':'sum'})
grants_grouped_by_uni.shape

(77, 1)

Command line 'voodoo' magic to get the list of cantons from the json file 

In [62]:
cantons =! cat "ch-cantons.topojson.json" | grep "id\":" | cut -d\" -f4

In [67]:
len(cantons)

26

Use Google API to find the location corresponding to the university

In [ ]:
google_key="0" #todo insert key
url = "https://maps.googleapis.com/maps/api/place/textsearch/xml?key="+google_key+"&query="
def get_city(row):
    if row.postalcode != None: 
        r = requests.get(url+row.University)
        place = BeautifulSoup(r.text, "xml")
        result = place.find('result')
        if result != None:
            city = result.find('formatted_address').text
            return city[city.index(',')+2:city.index(',')+6]
    
#grants_grouped_by_uni['postalcode'] = grants_grouped_by_uni.apply(get_city, axis=1)
#grants_grouped_by_uni.loc[grants_grouped_by_uni['Location Tag'] == "BE", 'postalcode'] = 3012
grants_grouped_by_uni.head()

TODO:

* we got 77 location tags and 26 cantons
* so the non-canton locations need to be mapped to cantons
* identify the 'how'
* also split NPO